In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from utils.convex_sets import Singleton, Polyhedron, CartesianProduct
from utils.convex_functions import SquaredTwoNorm
from utils.pwa_systems import PieceWiseAffineSystem, ShortestPathRegulator
from pydrake.all import MosekSolver, GurobiSolver

In [ ]:
# initial state
z1 = np.array([-3.5, .5, 0, 0])
q1 = z1[:2]

# target set
zK = np.array([3.5, 6.5, 0, 0])
qK = zK[:2]
Z = Singleton(zK)

# time horizon
K = 30

# cost matrices
q_dot_cost = .2 ** .5
Q = np.diag([0, 0, q_dot_cost, q_dot_cost])
R = np.eye(2)
S = Q # ininfluential
cost_matrices = (Q, R, S)

In [ ]:
# configuration bounds
Dq = [
    Polyhedron.from_bounds([-4, 0], [3, 1]),
    Polyhedron.from_bounds([-6, 1], [-5, 3]),
    Polyhedron.from_bounds([4, 1], [5, 2]),
    Polyhedron.from_bounds([-4, 3], [4, 4]),
    Polyhedron.from_bounds([-5, 5], [-4, 6]),
    Polyhedron.from_bounds([5, 4], [6, 6]),
    Polyhedron.from_bounds([-3, 6], [4, 7])
]

# velocity bounds
qdot_max = np.ones(2) * 1
qdot_min = - qdot_max
Dqdot = Polyhedron.from_bounds(qdot_min, qdot_max)

# control bounds
u_max = np.ones(2) * 1
u_min = - u_max
Du = Polyhedron.from_bounds(u_min, u_max)

# pwa domains
domains = [CartesianProduct((Dqi, Dqdot, Du)) for Dqi in Dq]

In [ ]:
# dynamics
A = np.array([
    [1, 0, 1, 0],
    [0, 1, 0, 1],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
])
B = np.vstack((np.zeros((2, 2)), np.eye(2)))
Bred = B / 10
c = np.zeros(4)
dynamics = [(A, Bred, c) if i in [1, 5] else (A, B, c) for i in range(len(domains))]

# pieceiwse affine system
pwa = PieceWiseAffineSystem(dynamics, domains)

In [ ]:
# select solver
# Mosek and Gurobi give different answers
# Gurobi's one is the correct one
solver = MosekSolver()

In [ ]:
# solve optimal control problem
relaxation = 0
reg = ShortestPathRegulator(pwa, K, z1, Z, cost_matrices, solver, relaxation)
sol = reg.solve()
print('Cost:', sol.spp.cost)
print('Solve time:', sol.spp.time)

# unpack result
q = sol.z[:, :2]
u = sol.u

In [ ]:
def plot_terrain(q=None, u=None):
    plt.rc('axes', axisbelow=True)
    plt.gca().set_aspect('equal')

    for i, Dqi in enumerate(Dq):
        color = 'lightcoral' if i in [1, 5] else 'lightcyan'
        Dqi.plot(facecolor=color)
        
    plt.scatter(*q1, s=300, c='g', marker='+', zorder=2)
    plt.scatter(*qK, s=300, c='g', marker='x', zorder=2)
    
    if q is not None:
        plt.plot(*q.T, c='k', marker='o', markeredgecolor='k', markerfacecolor='w')
        
        if u is not None:
            for t, ut in enumerate(u):
                plt.arrow(*q[t], *ut, color='b', head_starts_at_zero=0, head_width=.15, head_length=.3)
                
    plt.xlabel(r'$q_1, u_1$')
    plt.ylabel(r'$q_2, u_2$')
    plt.grid(1)

In [ ]:
# plot solution
plt.figure()
plot_terrain(q, u)

# Debug constraint satisfaction

In [ ]:
# rename variables in the spp solution
G = reg.spp.graph
phi = sol.spp.primal.phi
y = sol.spp.primal.y
z = sol.spp.primal.z
l = sol.spp.primal.l
x = sol.spp.primal.x

for k, e in enumerate(G.edges):
    
    # edges along the shortest path
    if np.isclose(phi[k], 1):
        
        # position of vertices incident with edge e
        xu, xv = x[G.vertex_indices(e)]
        
        # checks that xu - y[k] in (1 - phi[k]) Xu
        # since here phi[k] = 1, we must have xu = y[k]
        if not np.allclose(y[k], xu, atol=1.e-4):
            print('xu != y[k] at edge', k, e)
            print(np.round(y[k] - xu, 4))
            
        # checks that xv - z[k] in (1 - phi[k]) Xv
        # since here phi[k] = 1, we must have xv = z[k]
        if not np.allclose(z[k], xv, atol=1.e-4):
            print('xv != z[k] at edge', k, e)
            print(np.round(z[k] - xv, 4))
            
    else:
        
        # checks that y[k] in phi[k] Xu
        # since here phi[k] = 0, we must have y[k] = 0
        assert np.allclose(y[k], 0, atol=1.e-4)
        
        # checks that z[k] in phi[k] Xv
        # since here phi[k] = 0, we must have z[k] = 0
        assert np.allclose(z[k], 0, atol=1.e-4)

# What's wrong with the 401st edge?

In [ ]:
# select one of the linear constraints that enforce
# xu - y[k] in (1 - phi[k]) Xu
# for the 401st edge
c_violated = reg.spp.prog.linear_constraints()[4975]
print(str(c_violated))

# these should all be <= 1
print(phi[401] - y[401,4] + x[57,4])
print(phi[401] - y[401,5] + x[57,5])
print(phi[401] + y[401,4] - x[57,4])
print(phi[401] + y[401,5] - x[57,5])

In [ ]:
vars = np.concatenate((
    phi,
    y.flatten(),
    z.flatten(),
    l,
    x.flatten()
))
c_eval = c_violated.evaluator()
print('Satisfied:', c_eval.CheckSatisfied(vars))
print('Evaluated:', c_eval.Eval(vars))